In [1]:
import os
import random

import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import sparse

import stellargraph as sg
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import GraphSAGELinkGenerator
from stellargraph.layer import GraphSAGE, link_classification
from stellargraph.data import UniformRandomWalk
from stellargraph.data import UnsupervisedSampler
from sklearn.model_selection import train_test_split

import keras 
from sklearn import preprocessing, feature_extraction, model_selection
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score

from stellargraph import globalvar

Using TensorFlow backend.


### Loading the CORA network data

In [2]:
data_dir = "../stellargraph/data/cora"

Load the graph from edgelist.

In [3]:
edgelist = pd.read_csv(os.path.join(data_dir, "cora.cites"), sep='\t', header=None, names=["source", "target"])
edgelist["label"] = "cites"  # set the edge type

In [4]:
Gnx = nx.from_pandas_edgelist(edgelist, edge_attr="label")

Specify the node types.

In [5]:
nx.set_node_attributes(Gnx, "paper", "label")

Load the features and subject for the nodes.

In [6]:
feature_names = ["w_{}".format(ii) for ii in range(1433)]
column_names =  feature_names + ["subject"]
node_data = pd.read_csv(os.path.join(data_dir, "cora.content"), sep='\t', header=None, names=column_names)

The node features are all attributes except the paper subject (we will use this for plotting the embeddings later). Paper subject label are not used for unsupervised training.

In [7]:
node_features = node_data[feature_names]

Create a nodes test set

In [31]:
# This can be simplified once https://github.com/JuliaPy/pyjulia/issues/310 is fixed
from julia.api import LibJulia
api = LibJulia.load()
api.sysimage = "julia/sys.so"
api.init_julia()

from julia import Main
Main.eval('include("julia/dataset.jl")')

In [39]:
gtrain_edgelistpath, test_nodes, train_nodes = Main.Dataset.make_nodes_test_set(os.path.join(data_dir, "cora.cites"), 0.2)
gtrain_edgelist = pd.read_csv(gtrain_edgelistpath, sep=',', header=None, names=["source", "target"])
os.remove(gtrain_edgelistpath)
gtrain_edgelist["label"] = "cites"
Gtrain_nx = nx.from_pandas_edgelist(gtrain_edgelist, edge_attr="label")